<h1>F20BC Coursework</h1>
<p style="font-size:15px;">This notebook implements and analyses the Biologically-Inspired Computation coursework, focusing on Artificial Neural Networks (ANNs) and Particle Swarm Optimization (PSO). It includes a multi-layer ANN architecture trained using PSO to solve a regression task predicting concrete compressive strength. Additionally, it explores the impact of hyperparameters on model performance through systematic experiments.</p>

**<p style="font-size:18px;">Data Preparation</p>** 
This contains all the data preprocessing and cleaning</p> 

In [532]:
import os
print(os.getcwd())

/Users/imanteh/f20bc/F21BC/Notebooks


In [533]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [534]:
df = pd.read_csv("../Dataset/concrete_data.csv", skiprows=1)

In [535]:
df.head()

,cement,blast_furnace_slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age,concrete_compressive_strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [536]:
# Viewing the dimensionality of the dataset
df.shape

(1030, 9)

In [537]:
df.dropna(inplace = True)
df.isnull().sum()

cement                           0
blast_furnace_slag               0
fly_ash                          0
water                            0
superplasticizer                 0
coarse_aggregate                 0
fine_aggregate                   0
age                              0
concrete_compressive_strength    0
dtype: int64

In [538]:
print(f'Duplicate rows: {df.duplicated().sum()}')

Duplicate rows: 25


In [539]:
# Remove duplicate rows
df.drop_duplicates(inplace=True)

**<p style="font-size:18px;">Scaling and Encoding</p>** 

In [540]:
print(df.head())
print(df.columns)

   cement  blast_furnace_slag  fly_ash  water  superplasticizer  \
0   540.0                 0.0      0.0  162.0               2.5   
1   540.0                 0.0      0.0  162.0               2.5   
2   332.5               142.5      0.0  228.0               0.0   
3   332.5               142.5      0.0  228.0               0.0   
4   198.6               132.4      0.0  192.0               0.0   

   coarse_aggregate  fine_aggregate   age  concrete_compressive_strength  
0            1040.0            676.0   28                          79.99  
1            1055.0            676.0   28                          61.89  
2             932.0            594.0  270                          40.27  
3             932.0            594.0  365                          41.05  
4             978.4            825.5  360                          44.30  
Index(['cement', 'blast_furnace_slag', 'fly_ash', 'water', 'superplasticizer',
       'coarse_aggregate', 'fine_aggregate ', 'age',
       'concret

In [541]:
from sklearn.model_selection import train_test_split

X = df.iloc[:, :-1] # All columns except the last
y = df.iloc[:, -1] # The last column

In [542]:
print(X.shape)
print(y.shape)

(1005, 8)
(1005,)


In [543]:
print(X.head())

   cement  blast_furnace_slag  fly_ash  water  superplasticizer  \
0   540.0                 0.0      0.0  162.0               2.5   
1   540.0                 0.0      0.0  162.0               2.5   
2   332.5               142.5      0.0  228.0               0.0   
3   332.5               142.5      0.0  228.0               0.0   
4   198.6               132.4      0.0  192.0               0.0   

   coarse_aggregate  fine_aggregate   age  
0            1040.0            676.0   28  
1            1055.0            676.0   28  
2             932.0            594.0  270  
3             932.0            594.0  365  
4             978.4            825.5  360  


In [544]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [545]:
print("First 5 rows of scaled training data:")
print(X_train_scaled[:5])

First 5 rows of scaled training data:
[[-0.7036142   0.7362629  -0.8810702   0.1799759  -1.0226942   1.32341753
  -0.17765612 -0.62455318]
 [-0.83112259 -0.84582676  1.04788182 -0.72981642  0.63596651  1.35948653
   0.32102131  0.88513966]
 [ 0.43140366  1.06620495 -0.8810702   0.38479825 -0.18498677 -1.33280675
   0.00745899 -0.28365479]
 [-1.05233032  0.66864226  1.10026551 -0.30588178  0.28412939  0.42298068
  -0.33758549 -0.51092038]
 [-1.17404287 -0.84582676  1.31904441  0.5419875   0.50193332 -1.24005789
   1.18741037 -0.28365479]]


**<p style="font-size:18px;">Artificial Neural Network (ANN)</p>**
Below are the implementations of the ANN to train and predict concrete compressive strength

In [546]:
class activationFunction:
    def logisticFunction(x):
        return 1 / (1 + np.exp(-x))

    def reluFunction(x):
        return np.maximum(0, x)

    def hyperbolicFunction(x):
        return np.tanh(x)

    def leakyReLU(x, alpha=0.01):
        return np.maximum(x, alpha * x)

    def linearFunction(x):  # Identity function for regression output
        return x
    
    # def elu(x, alpha=1.0):
    #     return np.where(x > 0, x, alpha * (np.exp(x) - 1))

class ArtificialNeuralNetwork:
    def __init__(self, layerSize, activationFunction):
        self.layerSize = layerSize
        self.activationFunction = activationFunction
        self.weights = [np.random.randn(layer_sizes[i], layer_sizes[i + 1]) * np.sqrt(2 / layer_sizes[i]) for i in range(len(layer_sizes) - 1)]
        self.biases = [np.random.randn(1, layer_sizes[i + 1]) for i in range(len(layer_sizes) - 1)]

    def forwardPropagation(self, x):
        output = x
        for i in range(len(self.weights)):
            matrix_total = np.dot(output, self.weights[i]) + self.biases[i]
            output = self.activationFunction[i](matrix_total)  
        return output

Layer size is structure to be 8,16,8,1 because 
- 8 input layers
- 16 neurons in first hidden layer
- 8 for the second hidden layer
- 1 for the output layer<p>
This is to check the predicted values generated after forward propagation from the ANN.</p>

In [547]:
layer_sizes = [8, 16, 8, 1]

activation_functions = [
    activationFunction.logisticFunction,
    activationFunction.reluFunction,
    activationFunction.linearFunction,
]
ann = ArtificialNeuralNetwork(layer_sizes, activation_functions)

x_example = np.random.rand(8, 8)  #8 samples, 8 features
output = ann.forwardPropagation(x_example)
print("ANN Output (Sample Predictions):", output)

ANN Output (Sample Predictions): [[-2.2225788 ]
 [-2.19633093]
 [-2.13489117]
 [-1.89494876]
 [-2.08743616]
 [-2.43279361]
 [-2.5199721 ]
 [-2.36410738]]


**<p style="font-size:18px;">Loss Function</p>**
Since the problem domian is regression, MSE is utilised as the loss function

In [548]:
class lossFunction:
    def evaluate(self, y_pred, y_train):
        pass
class MeanSquaredError(lossFunction):
    def evaluate(self, y_pred, y_train):
        return np.mean((y_pred - y_train) ** 2)

In [549]:
y_train = y_train.to_numpy().reshape(-1, 1) # making y_train 2D
y_pred = ann.forwardPropagation(X_train_scaled) # Forward propagating  through the ANN

# checking y_pred and y_train has the correct shape
print("y_pred shape:", y_pred.shape) 
print("y_train shape:", y_train.shape)

# use to calculate the loss function
loss = loss_function.evaluate(y_pred, y_train)
print("Loss:", loss)

y_pred shape: (703, 1)
y_train shape: (703, 1)
Loss: 1648.9327764364702


**<p style="font-size:18px;">Implement PSO Algorithm</p>**

In [550]:
class Particle:
    def __init__(self,vectorSize):
            self.particlePosition=np.random.rand(vectorSize)
            self.particleVelocity=np.random.rand(vectorSize)
            self.bestPosition=np.copy(self.particlePosition)
            self.informants=[]

In [551]:
def particleToAnn(particle, annLayers, activationFunctions):
    neuralNetwork = ArtificialNeuralNetwork(layerSize=annLayers, activationFunction=activationFunctions)
    weightBiasIndexCount = 0
    for i in range(len(annLayers) - 1):
        prevValue = annLayers[i]
        nextValue = annLayers[i + 1]
        weightRange = prevValue * nextValue
        
        weight = particle.particlePosition[weightBiasIndexCount:weightBiasIndexCount + weightRange].reshape((prevValue, nextValue))
        weightBiasIndexCount += weightRange
        
        biases = particle.particlePosition[weightBiasIndexCount:weightBiasIndexCount + nextValue].reshape((1, nextValue))
        weightBiasIndexCount += nextValue
        
        # activation = activationFunctions[i]
        neuralNetwork.weights[i] = weight
        neuralNetwork.biases[i] = biases
    return neuralNetwork

In [552]:
def assessFitness(particle, dataset, annLayers, activationFunctions, lossFunction):
    x, y = dataset
    ann = particleToAnn(particle, annLayers, activationFunctions)
    predictions = ann.forwardPropagation(x)  # No transpose here
    loss = lossFunction.evaluate(predictions, y.reshape(-1, 1))
    return loss

In [553]:
class ParticleSwarmOptimisation:
    def __init__(self, swarmSize, alpha, beta, delta, omega, jumpSize, informantCount, vectorSize):
        self.swarmSize = swarmSize
        self.alpha = alpha
        self.beta = beta
        self.delta = delta
        self.omega = omega
        self.jumpSize = jumpSize
        self.informantCount = informantCount
        self.vectorSize = vectorSize
        self.global_best = None
        self.global_best_fitness = float('inf')

    def initInformants(self, informantCount, particleArray):
        for p in particleArray:
            potentialInformants = [potInf for potInf in particleArray if potInf != p]
            p.informants = [random.choice(potentialInformants) for _ in range(informantCount)]

    def get_best_informant(self, particle, dataset, annLayers, activationFunctions, lossFunction):
        bestInf = None
        bestFitnessInf = float('-inf')
        for i in particle.informants:
            fitness = assessFitness(i, dataset, annLayers, activationFunctions, lossFunction)
            if fitness > bestFitnessInf:
                bestFitnessInf = fitness
                bestInf = i
        return bestInf.particlePosition

    def psoOptimisation(self, swarmSize, alpha, beta, gamma, jumpSize, informantCount, vectorSize,
                        dataset, annLayers, activationFunctions, lossFunction, max_iterations=100):
        # Initialize particles
        particleArray = [Particle(vectorSize) for _ in range(swarmSize)]
        self.initInformants(informantCount, particleArray)

        best = None
        iteration = 0

        while iteration < max_iterations:
            # Update global best
            for p in particleArray:
                particleFitness = assessFitness(p, dataset, annLayers, activationFunctions, lossFunction)
                if best is None or particleFitness < assessFitness(best, dataset, annLayers, activationFunctions, lossFunction):
                    best = p

            for p in particleArray:
                previousBest = p.bestPosition
                informantsBest = self.get_best_informant(p, dataset, annLayers, activationFunctions, lossFunction)
                allBest = best.bestPosition

                b = np.random.uniform(0.0, beta)
                c = np.random.uniform(0.0, gamma)
                d = np.random.uniform(0.0, delta)

                updatedVelocity = (
                    alpha * p.particleVelocity +
                    b * (previousBest - p.particlePosition) +
                    c * (informantsBest - p.particlePosition) +
                    d * (allBest - p.particlePosition)
                )

                p.particleVelocity = updatedVelocity
                p.particlePosition += jumpSize * updatedVelocity

            iteration += 1

        return best.particlePosition


In [554]:
# Code to test out PSO functionality
X_train_scaled = np.random.rand(100, 8)  # Simulated scaled training features (100 samples, 8 features)
y_train = np.random.rand(100, 1)  # Simulated training labels (100 samples, 1 output)

# define hyperparameters for PSO 
swarmSize = 10
alpha = 0.5  # Inertia weight
beta = 0.5   # Cognitive parameter
delta = 0.5  # Social parameter
jumpSize = 0.5
informantCount = 3
vectorSize = sum([(annLayers[i] * annLayers[i + 1]) + annLayers[i + 1] for i in range(len(annLayers) - 1)])  # Total weights + biases
max_iterations = 50

# initialize the PSO optimizer
pso = ParticleSwarmOptimisation(
    swarmSize = swarmSize,
    alpha = alpha,
    beta = beta,
    delta = delta,
    omega = omega,
    jumpSize = jumpSize,
    informantCount = informantCount,
    vectorSize = vectorSize
)

# run it into the psoOptimization
best_parameters = pso.psoOptimisation(
    swarmSize = swarmSize,
    alpha = alpha,
    beta = beta,
    gamma = delta,  # Using delta as gamma as per your code
    jumpSize = jumpSize,
    informantCount = informantCount,
    vectorSize = vectorSize,
    
    dataset = (X_train_scaled, y_train),
    annLayers = annLayers,
    activationFunctions = activationFunctions,
    lossFunction = MeanSquaredError(),
    max_iterations = max_iterations
)
print("Optimized ANN Parameters:", best_parameters)

Optimized ANN Parameters: [0.37063912 0.85657189 0.71364493 0.89525127 0.72295771 0.72459837
 0.87796412 0.46690263 0.3764366  0.30163179 0.32721622 0.12581728
 0.10953848 0.47870397 0.33446195 0.58098229 0.43235957 0.58103131
 0.65918954 0.22166548 0.50339125 0.28006496 0.49545798 0.62956981
 0.37426486 0.6580594  0.94939656 0.61317097 0.4392104  0.86711004
 0.21082697 0.5462845  0.54813267 0.9297299  0.52988234 0.25358172
 0.19915285 0.71739453 0.15470174 0.31252236 0.53101414 0.40472971
 0.7659862  0.47804869 0.45129872 0.75487284 0.82833808 0.70697734
 0.8278776  0.43473115 0.09947274 0.82439813 0.56989926 0.07693777
 0.48818849 0.74107199 0.50456122 0.21133853 0.23317936 0.44011782
 0.71772847 0.50590969 0.77930805 0.38886539 0.2514252  0.6522236
 0.78898921 0.82624902 0.38974497 0.19924354 0.48856379 0.28597786
 0.85035908 0.5683979  0.85136676 0.50208929 0.7814206  0.21899166
 0.25350955 0.30980769 0.62983002 0.54761141 0.47579507 0.64438384
 0.3003716  0.30941264 0.34934108 0.8

<p style="font-size:18px;">Get Fitness Value</p>
The final fitness represents the performance of the ANN after its weights and biases have been optimized by the PSO

In [555]:
# Create a dummy particle and assign the optimized parameters
optimized_particle = Particle(vectorSize)
optimized_particle.particlePosition = best_parameters

# Convert the optimized particle position back to ANN weights and biases
optimized_ann = particleToAnn(
    optimized_particle,  # Pass the particle object
    annLayers,
    activationFunctions
)

# Assess the ANN with the optimized parameters
fitness_value = assessFitness(
    optimized_particle,  # Pass the particle object for fitness evaluation
    dataset = (X_train_scaled, y_train),
    annLayers = annLayers,
    activationFunctions = activationFunctions,
    lossFunction = MeanSquaredError()
)
print("Fitness Value:", fitness_value)

Fitness Value: 0.35703250820933297
